In [1]:
# !pip install ipython-autotime library to count run time
%load_ext autotime

time: 0 ns (started: 2021-09-16 15:16:24 +08:00)


In [2]:
import cv2
import os
from skimage.util import random_noise
from imwatermark import WatermarkEncoder
# from blind_watermark import WaterMark
from imwatermark import WatermarkDecoder
import numpy as np
import random
import glob
import pandas as pd 

time: 2.3 s (started: 2021-09-16 15:16:24 +08:00)


In [3]:
# this need to import seperately, else it will kernel die
from blind_watermark import WaterMark

bwm1 = WaterMark(password_img=1, password_wm=1)
bwm1.read_img('test/MRI_A.png')
wm = 'zhen'
bwm1.read_wm(wm, mode='str')

bwm1.embed('test/MRI_A.png')
len_wm = len(bwm1.wm_bit)
print('Put down the length of wm_bit {len_wm}'.format(len_wm=len_wm))

Put down the length of wm_bit 31
time: 2 s (started: 2021-09-16 15:16:26 +08:00)


# Part A: Encode

In [4]:
#Encode DWT A

infolder='in_img'
outfolder=os.path.join('out_img','algo_a')
in_img_list = os.listdir(infolder)
text='zhen'

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_A_'+img
    out_path = os.path.join(outfolder,out_name)
    
   
    encoder = WaterMark(password_img=1, password_wm=1)
    encoder.read_img(in_path)
    encoder.read_wm(text, mode='str')
    encoder.embed(out_path)
     
    print("Done Encoding: " + out_path)

Done Encoding: out_img\algo_a\Algo_A_MRI_A.png
Done Encoding: out_img\algo_a\Algo_A_MRI_B.png
Done Encoding: out_img\algo_a\Algo_A_MRI_C.png
Done Encoding: out_img\algo_a\Algo_A_MRI_D.png
Done Encoding: out_img\algo_a\Algo_A_MRI_E.png
Done Encoding: out_img\algo_a\Algo_A_MRI_F.png
Done Encoding: out_img\algo_a\Algo_A_MRI_G.png


KeyboardInterrupt: 

time: 16.6 s (started: 2021-09-16 15:16:28 +08:00)


In [ ]:
#Encode dwtDct B

infolder='in_img'
outfolder=os.path.join('out_img','algo_b')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()


for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_B_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDct')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


In [ ]:
#encode DwtDctSvd C

infolder='in_img'
outfolder=os.path.join('out_img','algo_c')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_C_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDctSvd')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


In [ ]:
#encode RivaGAN D

infolder='in_img'
outfolder=os.path.join('out_img','algo_d')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_D_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    encoder.loadModel()
    img_encoded = encoder.encode(in_img,'rivaGan')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


# Part B: Attacking

In [7]:
#Attacking Methods

def gaussian(img, output_file_name):
    img_shape = img.shape
    gauss_img = random_noise(img, mode='gaussian', mean=0, var=0.01, clip= True)
    gauss_img = np.array(255*gauss_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, gauss_img)

def salt_pepper(img, output_file_name):
    ratio = 0.1 #Probability of the noise
    #0.01 all pass?
    #0.1 algorithm d only partial pass
    snp_img = np.zeros(img.shape,np.uint8)
    thres = 1 - ratio
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rdn = random.random()
            if rdn < ratio:
                snp_img[i][j] = 0
            elif rdn > thres:
                snp_img[i][j] = 255
            else:
                snp_img[i][j] = img[i][j]
    cv2.imwrite(output_file_name, snp_img)
    
def speckle(img, output_file_name):
    speckle_img = random_noise(img, mode='speckle', mean=0, var=0.01, clip= True)
    # The above function returns a floating-point image on the range [0, 1], thus we changed it to 'uint8' and from [0,255]
    speckle_img = np.array(255*speckle_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, speckle_img)
    
def poisson(img, output_file_name):
    img_shape = img.shape
    noise = np.random.poisson(20, img_shape)
    #30 all fail 
    output = img + noise
    cv2.imwrite(output_file_name, output)
    
def rotate(img, output_file_name, angle=10):
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D(center=(cols / 2, rows / 2), angle=angle, scale=1)
    output_img = cv2.warpAffine(img, M, (cols, rows))
    cv2.imwrite(output_file_name, output_img)
    
def upscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=1.25, fy=1.25, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def downscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=0.75, fy=0.75, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def averaging(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.blur(img,(5,5)) 
    cv2.imwrite(output_file_name, output_img)
    
def gaussian_blurring(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.GaussianBlur(img,(5,5),0) 
    cv2.imwrite(output_file_name, output_img)
    
def median_blurring(img, output_file_name):
    output_img = cv2.medianBlur(img,7)
    #5 all pass
    cv2.imwrite(output_file_name, output_img)
    
def bilateral_filtering(img, output_file_name):
    output_img = cv2.bilateralFilter(img,9,75,75)
    cv2.imwrite(output_file_name, output_img)
    
def sharpen_filtering(img, output_file_name):
    sharpen_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharped_img = cv2.filter2D(img, -1, sharpen_filter)
    cv2.imwrite(output_file_name, sharped_img)

def crop_horizontal(img, output_file_name):
    ratio = 0.5  
    #left 50% all pass
    img_shape = img.shape
    height = int(img_shape[0] * ratio)
    horizontal_img = img[:height, :, :]
    cv2.imwrite(output_file_name, horizontal_img)

def crop_vertical(img, output_file_name):
    ratio = 0.5
    img_shape = img.shape
    vertical = int(img_shape[1] * ratio)
    vertical_img = img[:, :vertical, :]
    cv2.imwrite(output_file_name, vertical_img)

def increase_brightness(img, output_file_name):
    ratio = 1.4 # 
    #+10% all pass
    #+50% algorithm D
    inc_bright_img = img * ratio
    inc_bright_img[inc_bright_img > 255] = 255
    cv2.imwrite(output_file_name, inc_bright_img)
    
def decrease_brightness(img, output_file_name):
    ratio = 0.6 
    # -50% brightness all pass
    dec_bright_img = img * ratio
    dec_bright_img[dec_bright_img > 255] = 255
    cv2.imwrite(output_file_name, dec_bright_img)
    
def masks(img, output_file_name):
    n = 5 # Set 2 mask
    ratio = 0.3 # mask ratio
    input_img_shape = img.shape
    mask_img = img.copy()
    for i in range(n):
        tmp = np.random.rand() * (1 - ratio)  # random one place to put mask，1-ratio to avoid overfloat
        start_height, end_height = int(tmp * input_img_shape[0]), int((tmp + ratio) * input_img_shape[0])
        tmp = np.random.rand() * (1 - ratio)
        start_width, end_width = int(tmp * input_img_shape[1]), int((tmp + ratio) * input_img_shape[1])

        mask_img[start_height:end_height, start_width:end_width, :] = 0
    cv2.imwrite(output_file_name, mask_img)
    
def init_attack(input_image, image_name):
    print('Attacking ' + image_name)
    
    a = image_name.split('.')[0]
    mri_name = a.split('_')[2]+'_'+a.split('_')[3]
    algo_name = a.split('_')[0]+'_'+a.split('_')[1]
    
    
    output_path=os.path.join('out_att',mri_name,algo_name.lower())
#     print(output_path)
    
    gaussian(input_image, os.path.join(output_path,'Gaussian_Noise.png'))
    salt_pepper(input_image, os.path.join(output_path,'Salt_Pepper.png'))
    speckle(input_image, os.path.join(output_path,'Speckle.png'))
    poisson(input_image, os.path.join(output_path,'Poisson.png'))
    rotate(input_image, os.path.join(output_path,'Rotate.png'), angle=10)
    upscale(input_image, os.path.join(output_path,'ScaleUp.png'))
    downscale(input_image, os.path.join(output_path,'ScaleDown.png'))
    averaging(input_image, os.path.join(output_path,'Averaging.png'))
    gaussian_blurring(input_image, os.path.join(output_path,'Gaussian_Blurring.png'))
    median_blurring(input_image, os.path.join(output_path,'Median_Blurring.png'))
    bilateral_filtering(input_image, os.path.join(output_path,'Bilateral_Filtering.png'))
    sharpen_filtering(input_image, os.path.join(output_path,'Sharpen_Filtering.png'))
    crop_horizontal(input_image, os.path.join(output_path,'Crop_Horizontal.png'))
    crop_vertical(input_image, os.path.join(output_path,'Crop_Vertical.png'))
    increase_brightness(input_image, os.path.join(output_path,'Brightness_Increase.png'))
    decrease_brightness(input_image, os.path.join(output_path,'Brightness_Decrease.png'))
    masks(input_image, os.path.join(output_path,'Masks.png'))
    cv2.imwrite(os.path.join(output_path,'JPG.jpg'), input_image)


time: 0 ns (started: 2021-09-16 15:17:34 +08:00)


In [8]:
infolder=os.path.join('out_img')
outfolder=os.path.join('out_att')

img_folder_list = os.listdir(infolder) 

for folder in img_folder_list: 
#     print(folder)
    in_img_list = os.listdir(os.path.join(infolder,folder)) 
    
    for img_path in in_img_list: 
#         print(img_path)
        img = cv2.imread(os.path.join(infolder,folder,img_path))
        init_attack(img,img_path)   
        

Attacking Algo_A_MRI_A.png
Attacking Algo_A_MRI_B.png
Attacking Algo_A_MRI_C.png
Attacking Algo_A_MRI_D.png
Attacking Algo_A_MRI_E.png
Attacking Algo_A_MRI_F.png
Attacking Algo_A_MRI_G.png
Attacking Algo_A_MRI_H.png
Attacking Algo_A_MRI_I.png
Attacking Algo_A_MRI_J.png
Attacking Algo_A_MRI_K.png
Attacking Algo_A_MRI_L.png
Attacking Algo_A_MRI_M.png
Attacking Algo_A_MRI_N.png
Attacking Algo_A_MRI_O.png
Attacking Algo_A_MRI_P.png
Attacking Algo_A_MRI_Q.png
Attacking Algo_A_MRI_R.png
Attacking Algo_A_MRI_S.png
Attacking Algo_A_MRI_T.png
Attacking Algo_B_MRI_A.png
Attacking Algo_B_MRI_B.png
Attacking Algo_B_MRI_C.png
Attacking Algo_B_MRI_D.png
Attacking Algo_B_MRI_E.png
Attacking Algo_B_MRI_F.png
Attacking Algo_B_MRI_G.png
Attacking Algo_B_MRI_H.png
Attacking Algo_B_MRI_I.png
Attacking Algo_B_MRI_J.png
Attacking Algo_B_MRI_K.png
Attacking Algo_B_MRI_L.png
Attacking Algo_B_MRI_M.png
Attacking Algo_B_MRI_N.png
Attacking Algo_B_MRI_O.png
Attacking Algo_B_MRI_P.png
Attacking Algo_B_MRI_Q.png
A

# Part C: Decode

In [10]:
def algo_A(image_list, result):
    count = 1
    print('ALGORITHM A')
    print('===========')
    for img in image_list:
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            bwm1=WaterMark(password_img=1,password_wm=1)
            output = bwm1.extract(img, wm_shape=31, mode='str')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')

    
def algo_B(image_list, result):
    count = 1
    print('ALGORITHM B')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            watermark = decoder.decode(bgr, 'dwtDct')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
def algo_C(image_list, result):
    count = 1
    print('ALGORITHM C')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            watermark = decoder.decode(bgr, 'dwtDctSvd')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
def algo_D(image_list, result):
    count = 1
    print('ALGORITHM D')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            decoder.loadModel()
            watermark = decoder.decode(bgr, 'rivaGan')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
              
                


time: 0 ns (started: 2021-09-16 15:19:14 +08:00)


In [11]:
def Decoder(df):
    img_folder_list = os.listdir('out_att') # Check all folder in this dic

    for folder in img_folder_list: 
        algo_list = os.listdir(os.path.join('out_att',folder)) # Check All algo folder
        
        for algo in algo_list: 
            imdir = os.path.join('out_att',folder,algo,'')
            ext = ['png', 'jpg'] 
            result = []
            result.append(folder)
            files = []
            [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
            image_list = [(file) for file in files]

            if algo == 'algo_a':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_A(image_list, result)
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_b':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_B(image_list, result)            
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_c':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_C(image_list, result)  
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_d':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_D(image_list, result) 
                print(result)
                df_length = len(df)
                df.loc[df_length] = result  

time: 0 ns (started: 2021-09-16 15:19:14 +08:00)


In [14]:
#initiate decode

title = ['Photo', 'Algorithm', 'Averaging', 'BilateralFiltering', 'BrightnessDecrease',
        'BrightnessIncrease', 'CropHorizontal', 'CropVertical', 'GaussianBlurring', 
        'GaussianNoise', 'Masks', 'MedianBlurring', 'Poisson', 'Rotate', 'SaltPeper',
        'ScaleDown', 'ScaleUp', 'SharpenFiltering', 'Speckle','JPG']

df = pd.DataFrame(columns=title)

Decoder(df)

output_path=os.path.join('csv','result2.csv')

df.to_csv(output_path, index = False)

[Photo: MRI_A]
ALGORITHM A
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: Error
Result      : Fail
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text:  @ 
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: Error
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text: Error
Result      : Fail
-----
15: ScaleUp
Decoded text: Error
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----


Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text:     
Result      : Fail
-----
15: ScaleUp
Decoded text:    
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_B', 'algo_c', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass']
[Photo: MRI_B]
ALGORITHM D
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : 

Decoded text: zhen
Result      : Pass
-----
['MRI_D', 'algo_a', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass']
[Photo: MRI_D]
ALGORITHM B
1: Averaging
Decoded text:     
Result      : Fail
-----
2: BilateralFiltering
Decoded text:     
Result      : Fail
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text:     
Result      : Fail
-----
8: GaussianNoise
Decoded text: Error
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text:     
Result      : Fail
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen


Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: zhen
Result      : Pass
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: zhen
Result      : Pass
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: z`en
Result      : Fail
-----
13: SaltPepper
Decoded text: xle&
Result      : Fail
-----
14: ScaleDown
Decoded text: zhen
Result      : Pass
-----
15: ScaleUp
Decoded text: zhen
Result      : Pass
-----
16: SharpenFiltering
Decoded text: zhen
Result      : Pass
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_E', 'algo_d', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']
[Photo: MRI_F]
ALGORITHM A
1: Aver

Decoded text:     
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text:     
Result      : Fail
-----
['MRI_G', 'algo_b', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail']
[Photo: MRI_G]
ALGORITHM C
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: xhen
Result      : Fail
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: Error
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: z

...

Decoded text: z`en
Result      : Fail
-----
13: SaltPepper
Decoded text: h`%&
Result      : Fail
-----
14: ScaleDown
Decoded text: z`en
Result      : Fail
-----
15: ScaleUp
Decoded text: zhen
Result      : Pass
-----
16: SharpenFiltering
Decoded text: zhen
Result      : Pass
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_M', 'algo_d', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Pass']
[Photo: MRI_N]
ALGORITHM A
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: Error
Result      : Fail
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
R

Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: Error
Result      : Fail
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: Error
Result      : Fail
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text:     
Result      : Fail
-----
15: ScaleUp
Decoded text:     
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      

Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: Error
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text: Error
Result      : Fail
-----
15: ScaleUp
Decoded text: Error
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text: zhen
Result      : Pass
-----
['MRI_Q', 'algo_a', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Pass']
[Photo: MRI_Q]
ALGORITHM B
1: Averaging
Decoded text:     
Result      : Fail
-----
2: BilateralFiltering
Decoded text:     
Result      : Fail
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      

Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: zhen
Result      : Pass
-----
7: GaussianBlurring
Decoded text: zhen
Result      : Pass
-----
8: GaussianNoise
Decoded text: zhen
Result      : Pass
-----
9: Masks
Decoded text: zhen
Result      : Pass
-----
10: MedianBlurring
Decoded text: zhen
Result      : Pass
-----
11: Poisson
Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text: z`en
Result      : Fail
-----
13: SaltPepper
Decoded text: zle&
Result      : Fail
-----
14: ScaleDown
Decoded text: zhen
Result      : Pass
-----
15: ScaleUp
Decoded text: zhen
Result      : Pass
-----
16: SharpenFiltering
Decoded text: zhen
Result      : Pass
-----
17: Speckle
Decoded text: zhen
Result      : P

Decoded text: zhen
Result      : Pass
-----
12: Rotate
Decoded text:     
Result      : Fail
-----
13: SaltPepper
Decoded text: zhen
Result      : Pass
-----
14: ScaleDown
Decoded text:     
Result      : Fail
-----
15: ScaleUp
Decoded text: 
Result      : Fail
-----
16: SharpenFiltering
Decoded text: Error
Result      : Fail
-----
17: Speckle
Decoded text: zhen
Result      : Pass
-----
18: JPG
Decoded text:   
Result      : Fail
-----
['MRI_T', 'algo_b', 'Fail', 'Fail', 'Pass', 'Pass', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail', 'Fail', 'Fail', 'Pass', 'Fail']
[Photo: MRI_T]
ALGORITHM C
1: Averaging
Decoded text: zhen
Result      : Pass
-----
2: BilateralFiltering
Decoded text: zhen
Result      : Pass
-----
3: BrightnessDecrease
Decoded text: zhen
Result      : Pass
-----
4: BrightnessIncrease
Decoded text: zhen
Result      : Pass
-----
5: CropHorizontal
Decoded text: zhen
Result      : Pass
-----
6: CropVertical
Decoded text: zhzh
Result      :

In [15]:
df

,Photo,Algorithm,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
0,MRI_A,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
1,MRI_A,algo_b,Fail,Fail,Pass,Pass,Pass,Fail,Fail,Fail,Pass,Fail,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Fail
2,MRI_A,algo_c,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
3,MRI_A,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass
4,MRI_B,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Fail,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MRI_S,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass
76,MRI_T,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
77,MRI_T,algo_b,Fail,Fail,Pass,Pass,Pass,Fail,Fail,Fail,Pass,Fail,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Fail
78,MRI_T,algo_c,Pass,Pass,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass


time: 62 ms (started: 2021-09-16 15:49:10 +08:00)


# PSNR

In [1]:
import os
import cv2
import pandas as pd
from math import log10, sqrt
import numpy as np

In [2]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

### ORI VS ENCODED

In [3]:
folder_1 = os.path.join('in_img')
folder_2 = os.path.join('out_img')

original_img = os.listdir(folder_1)
encoded_img = os.listdir(folder_2)

In [4]:
list_1 = []
for item in original_img:
    img1 = cv2.imread(os.path.join(folder_1,item))
    list_1.append(img1)
   
list_2 = []
for folder in encoded_img: 
    img_list = os.listdir(os.path.join(folder_2,folder)) 
    
    for img_path in img_list: 
        img2 = cv2.imread(os.path.join(folder_2,folder,img_path))
        list_2.append(img2)
        
encoded_A = list_2[0:20]
encoded_B = list_2[20:40]
encoded_C = list_2[40:60]
encoded_D = list_2[60:80]

In [5]:
length = len(list_1)

list_M1_1 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_A[x]
    psnr_score_m1_1 = PSNR(img1, img2)
    list_M1_1.append(psnr_score_m1_1)
    
list_M1_2 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_B[x]
    psnr_score_m1_2 = PSNR(img1, img2)
    list_M1_2.append(psnr_score_m1_2)
    
list_M1_3 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_C[x]
    psnr_score_m1_3 = PSNR(img1, img2)
    list_M1_3.append(psnr_score_m1_3)
    
list_M1_4 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_D[x]
    psnr_score_m1_4 = PSNR(img1, img2)
    list_M1_4.append(psnr_score_m1_4)

In [6]:
df_M1 = pd.DataFrame(index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                            'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                            'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                            'MRI_S', 'MRI_T'])
df_M1['Encoded Algo A'] = list_M1_1
df_M1['Encoded Algo B'] = list_M1_2
df_M1['Encoded Algo C'] = list_M1_3
df_M1['Encoded Algo D'] = list_M1_4

df_M1

,Encoded Algo A,Encoded Algo B,Encoded Algo C,Encoded Algo D
MRI_A,35.235558,43.736332,46.975832,40.407926
MRI_B,35.264241,43.673936,46.941056,40.392352
MRI_C,35.236542,43.739201,47.001208,40.414981
MRI_D,35.248119,43.633610,46.921730,40.424273
MRI_E,35.256293,43.607840,46.924854,40.424904
MRI_F,35.483483,42.822696,44.768114,40.487141
MRI_G,35.179816,43.585932,46.897594,40.412936
MRI_H,35.272499,43.666232,46.937527,40.430422
MRI_I,35.185611,43.662848,46.975369,40.434533
MRI_J,35.185798,43.600485,46.919906,40.447968


### ENCODED VS ATTACKED 

In [7]:
folder_3 = os.path.join('out_img')
folder_4 = os.path.join('out_att')

encoded_img = os.listdir(folder_3)
attacked_img = os.listdir(folder_4)

In [8]:
list_3 = []
for folder in encoded_img: 
    img_list = os.listdir(os.path.join(folder_3,folder)) 
    
    for img_path in img_list: 
        img3 = cv2.imread(os.path.join(folder_3,folder,img_path))
        list_3.append(img3)
        
encoded_A = list_3[0:20]
encoded_B = list_3[20:40]
encoded_C = list_3[40:60]
encoded_D = list_3[60:80]

list_4 = []
for folder in attacked_img: 
    img_list = os.listdir(os.path.join(folder_4,folder)) 
    
    for infolder in img_list:
        in_list = os.listdir(os.path.join(folder_4,folder,infolder))
    
        for img_path in in_list: 
            img4 = cv2.imread(os.path.join(folder_4,folder,infolder,img_path))
            list_4.append(img4)

### Algo A

In [9]:
#algorithm A

Algorithm_att_a = []
a_psnr_list = []
x=0

for y in encoded_A: #20 combination
    start=x+(x*71)
    end=18+(x*72)
#     print(start,end)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            a_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            a_psnr_list.append('NaN')
    x=x+1

In [10]:
n = 20
list_M2_1 = []
len_1 = len(a_psnr_list)
for i in range(n):
    start = int(i*len_1/n)
    end = int((i+1)*len_1/n)
    list_M2_1.append(a_psnr_list[start:end])

In [11]:
df_M2_A = pd.DataFrame(list_M2_1, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_A

,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
MRI_A,32.117124,32.754567,28.241812,27.994443,NaN,NaN,34.813590,28.423508,41.303914,32.569778,31.954123,27.520225,28.237683,34.820290,NaN,NaN,28.314520,29.986017
MRI_B,31.799591,32.518904,28.139525,27.934750,NaN,NaN,34.518200,28.412822,41.235844,33.326550,31.634789,27.500321,28.086630,34.853299,NaN,NaN,28.228897,30.008529
MRI_C,32.520081,33.063695,28.803061,27.986568,NaN,NaN,35.222158,28.408779,41.298685,33.138822,32.267207,27.496299,28.294484,34.756488,NaN,NaN,28.319291,30.531229
MRI_D,32.183493,32.706348,28.528106,27.931044,NaN,NaN,34.960652,28.413525,41.237652,33.504550,31.909111,27.485806,28.233314,34.815738,NaN,NaN,28.255875,30.453501
MRI_E,32.410746,32.811867,29.082715,27.644648,NaN,NaN,35.184670,28.407270,41.285853,34.778170,32.069170,27.470396,28.263903,34.746681,NaN,NaN,28.278760,31.063833
MRI_F,33.328137,33.527773,31.188261,28.085587,NaN,NaN,35.999152,28.407901,40.938221,31.833293,32.908823,27.460522,28.848092,34.656605,NaN,NaN,28.447180,32.965949
MRI_G,34.033543,33.934502,28.451976,27.767666,NaN,NaN,37.507623,28.390346,41.313855,32.873411,33.388422,27.458357,28.430778,34.843076,NaN,NaN,28.501737,30.500129
MRI_H,33.314117,33.268639,28.643223,27.720896,NaN,NaN,36.877036,28.403499,41.312534,32.016685,32.650297,27.467414,28.282381,34.680257,NaN,NaN,28.390956,30.366965
MRI_I,33.656610,33.434497,28.755042,27.690760,NaN,NaN,37.258478,28.391534,41.345052,33.072296,32.857229,27.462899,28.216085,34.780999,NaN,NaN,28.419109,30.715808
MRI_J,34.415676,34.132262,29.796815,28.214077,NaN,NaN,37.361116,28.395488,41.266095,32.103059,33.746502,27.457462,29.009338,34.667115,NaN,NaN,28.470619,31.353447


### Algo B

In [12]:
#algorithm B
    
Algorithm_att_b = []
b_psnr_list = []
x=0

for y in encoded_B: #20 combination
    start=18+(x*72)
    end=36+(x*72)
#     print(start,end)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            b_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            b_psnr_list.append('NaN')
    x=x+1

In [13]:
n = 20
list_M2_2 = []
len_2 = len(b_psnr_list)
for i in range(n):
    start = int(i*len_2/n)
    end = int((i+1)*len_2/n)
    list_M2_2.append(b_psnr_list[start:end])

In [14]:
df_M2_B = pd.DataFrame(list_M2_2, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_B

,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
MRI_A,32.331482,33.025608,28.228999,27.994902,NaN,NaN,35.019968,28.419366,44.314787,32.300039,32.215374,27.517032,28.259544,34.839443,NaN,NaN,28.450205,30.008688
MRI_B,32.005746,32.769457,28.130901,27.949941,NaN,NaN,34.736317,28.410804,44.254120,33.251179,31.867089,27.499587,28.079076,34.874958,NaN,NaN,28.316204,30.036769
MRI_C,32.794409,33.422151,28.787304,28.026005,NaN,NaN,35.462038,28.405528,44.325786,32.487520,32.598040,27.493111,28.315062,34.766548,NaN,NaN,28.475830,30.563131
MRI_D,32.423699,32.996241,28.523081,27.949432,NaN,NaN,35.209386,28.409849,44.244014,32.720353,32.180436,27.485208,28.256861,34.888377,NaN,NaN,28.385082,30.480880
MRI_E,32.663958,33.113085,29.159092,27.673616,NaN,NaN,35.414886,28.406500,44.295845,32.157713,32.356961,27.470048,28.287780,34.780997,NaN,NaN,28.384122,31.117004
MRI_F,33.742334,34.024879,31.270753,28.195443,NaN,NaN,36.254654,28.394466,42.941381,31.599473,33.388792,27.457866,28.976289,34.637979,NaN,NaN,28.644218,33.026530
MRI_G,34.591630,34.512368,28.461471,27.747699,NaN,NaN,37.936606,28.389104,44.282992,32.530127,33.968876,27.464506,28.467799,34.803907,NaN,NaN,28.737853,30.545871
MRI_H,33.738913,33.693648,28.668441,27.714949,NaN,NaN,37.280838,28.403593,44.297370,32.741052,33.045036,27.470141,28.308088,34.687975,NaN,NaN,28.554788,30.407487
MRI_I,34.127065,33.892911,28.789161,27.687090,NaN,NaN,37.680630,28.393035,44.369075,32.599498,33.304726,27.454504,28.221485,34.746537,NaN,NaN,28.601184,30.765115
MRI_J,35.040912,34.787967,29.777701,28.299000,NaN,NaN,37.783767,28.391768,44.253398,32.300547,34.393283,27.451566,29.147752,34.688299,NaN,NaN,28.718932,31.401462


### Algo C

In [15]:
#algorithm C
    
Algorithm_att_c = []
c_psnr_list = []
x=0

for y in encoded_C: #20 combination
    start=36+(x*72)
    end=54+(x*72)
#     print(start,end)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            c_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            c_psnr_list.append('NaN')
    x=x+1

In [16]:
n = 20
list_M2_3 = []
len_3 = len(c_psnr_list)
for i in range(n):
    start = int(i*len_3/n)
    end = int((i+1)*len_3/n)
    list_M2_3.append(c_psnr_list[start:end])

In [17]:
df_M2_C = pd.DataFrame(list_M2_3, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_C

,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
MRI_A,32.474119,33.194787,28.230715,27.987514,NaN,NaN,35.355393,28.412533,49.723238,32.374806,32.330000,27.512739,28.263060,34.816100,NaN,NaN,28.480889,30.020243
MRI_B,32.134906,32.928023,28.135628,27.944822,NaN,NaN,35.050179,28.407853,50.382826,32.218816,31.972597,27.498722,28.081391,34.840060,NaN,NaN,28.343976,30.042539
MRI_C,32.971979,33.615904,28.793074,28.020607,NaN,NaN,35.863243,28.403661,50.467233,32.386269,32.727855,27.490861,28.321786,34.766568,NaN,NaN,28.511968,30.577866
MRI_D,32.577179,33.163271,28.527916,27.941108,NaN,NaN,35.583828,28.411451,50.414008,32.268372,32.293471,27.487572,28.262666,34.866653,NaN,NaN,28.415210,30.497159
MRI_E,32.829597,33.279541,29.171702,27.669745,NaN,NaN,35.809741,28.396500,49.621318,33.284953,32.477054,27.473879,28.295203,34.748381,NaN,NaN,28.414539,31.113507
MRI_F,33.976378,34.233225,31.286311,28.193807,NaN,NaN,36.770655,28.404697,46.428265,32.081442,33.534786,27.456980,28.980826,34.600903,NaN,NaN,28.695678,33.051972
MRI_G,34.885389,34.765950,28.466760,27.742428,NaN,NaN,38.795900,28.390866,49.587892,31.738662,34.163860,27.457343,28.474048,34.802548,NaN,NaN,28.803197,30.549579
MRI_H,33.973519,33.898880,28.674612,27.708576,NaN,NaN,37.968913,28.399979,50.208415,32.243851,33.207988,27.462908,28.310055,34.725655,NaN,NaN,28.603570,30.414749
MRI_I,34.392710,34.105529,28.794566,27.685308,NaN,NaN,38.452824,28.393418,49.769111,31.957302,33.469738,27.456645,28.223362,34.758597,NaN,NaN,28.640754,30.770883
MRI_J,35.383876,35.062821,29.782941,28.294828,NaN,NaN,38.605682,28.392652,50.323207,31.257856,34.602686,27.456695,29.159451,34.679416,NaN,NaN,28.770259,31.407914


### Algo D

In [18]:
#algorithm D

Algorithm_att_d = []
d_psnr_list = []
x=0

for y in encoded_D: #20 combination
    start=54+(x*72)
    end=72+(x*72)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            d_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            d_psnr_list.append('NaN')
    x=x+1

In [19]:
n = 20
list_M2_4 = []
len_4 = len(d_psnr_list)
for i in range(n):
    start = int(i*len_4/n)
    end = int((i+1)*len_4/n)
    list_M2_4.append(d_psnr_list[start:end])

In [20]:
df_M2_D = pd.DataFrame(list_M2_4, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_D

,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
MRI_A,32.389825,33.004342,28.239081,27.966556,NaN,NaN,35.240855,28.420460,43.458634,32.911970,32.192997,27.509461,28.251523,34.817963,NaN,NaN,28.416257,30.035206
MRI_B,32.057409,32.763584,28.140835,27.912355,NaN,NaN,34.943104,28.402359,43.425948,32.233397,31.857077,27.497587,28.078587,34.881486,NaN,NaN,28.304721,30.052502
MRI_C,32.865132,33.400716,28.798988,28.004245,NaN,NaN,35.732969,28.411475,43.506855,32.249897,32.569495,27.490997,28.300827,34.738852,NaN,NaN,28.449375,30.585078
MRI_D,32.498488,32.984625,28.537588,27.932001,NaN,NaN,35.459053,28.397265,43.477438,32.251825,32.162407,27.490593,28.250092,34.886950,NaN,NaN,28.370381,30.515257
MRI_E,32.739044,33.087672,29.207001,27.657219,NaN,NaN,35.682129,28.400572,43.490147,33.301323,32.341048,27.472639,28.275680,34.751365,NaN,NaN,28.373007,31.146095
MRI_F,33.837782,33.971042,31.327527,28.219484,NaN,NaN,36.659266,28.407002,42.788893,31.879875,33.325470,27.459109,28.905780,34.634693,NaN,NaN,28.631611,33.080165
MRI_G,34.693765,34.437013,28.478504,27.723433,NaN,NaN,38.532639,28.394439,43.557708,31.828158,33.892204,27.459629,28.445683,34.841580,NaN,NaN,28.704062,30.570710
MRI_H,33.821229,33.639548,28.697494,27.715298,NaN,NaN,37.740991,28.403674,43.568790,32.843924,33.013932,27.461006,28.297852,34.695264,NaN,NaN,28.534200,30.434517
MRI_I,34.218929,33.833771,28.822611,27.688641,NaN,NaN,38.210295,28.390454,43.540140,32.558359,33.245473,27.458140,28.229015,34.762289,NaN,NaN,28.581348,30.794604
MRI_J,35.154814,34.705729,29.767278,28.303079,NaN,NaN,38.347381,28.394181,43.468655,32.751359,34.308725,27.461115,29.081830,34.712705,NaN,NaN,28.669786,31.433053
